In [1]:
from qiskit import *
from qiskit.tools.visualization import plot_histogram
import numpy as np

Using the NOT gate (expressed as x in Qiskit), the CNOT gate (expressed as cx in Qiskit) and the Toffoli gate (expressed as ccx in Qiskit) create functions to implement the XOR, AND, NAND, OR and FANOUT gates.

An implementation of the NOT gate is provided as an example.

## NOT gate 
This function takes a binary string input ('0' or '1') and returns the opposite binary output'.

In [2]:
def NOT(input):
    """
    Takes as input a string representing either a binary 0 or 1,
    returns its binary complement.
    
    input (string): a binary number representing a bit.
    returns(int): the complement of the input bit.
    """
    q = QuantumRegister(1) # a qubit in which to encode and manipulate the input
    c = ClassicalRegister(1) # a bit to store the output
    qc = QuantumCircuit(q, c) # this is where the quantum program goes
    
    # We encode '0' as the qubit state |0⟩, and '1' as |1⟩
    # Since the qubit is initially |0⟩, we don't need to do anything for an input of '0'
    # For an input of '1', we do an x to rotate the |0⟩ to |1⟩
    if input=='1':
        qc.x( q[0] )
        
    # Now we've encoded the input, we can do a NOT on it using x
    qc.x( q[0] )
    
    # Finally, we extract the |0⟩/|1⟩ output of the qubit and encode it in the bit c[0]
    qc.measure( q[0], c[0] )
    
    # We'll run the program on a simulator
    backend = Aer.get_backend('qasm_simulator')
    # Since the output will be deterministic, we can use just a single shot to get it
    job = execute(qc,backend,shots=1)
    output = next(iter(job.result().get_counts()))
    
    return output

## XOR gate 
Takes two binary strings as input and gives one as output.

The output is '0' when the inputs are equal and '1' otherwise.

The following is the logic table for the CNOT gate, we can see that the result that is stored in |1⟩ is in fact the result we are interested in, so we apply the cx gate on the two inputs, and measure |1⟩, which is where CNOT makes its changes if the first qubit is not |0⟩

<table>
  <tr>
    <th colspan="2">Input</th>
    <th colspan="2">CNOT Output</th>
  </tr>
  <tr>
      <td>q<sub>0</sub></td>
    <td >q<sub>1</sub></td>
    <td >q<sub>0</sub></td>
    <td >q<sub>1</sub></td>
  </tr>
  <tr>
    <td >0</td>
    <td >0</td>
    <td >0</td>
    <td >0</td>
  </tr>
  <tr>
    <td >0</td>
    <td >1</td>
    <td >0</td>
    <td >1</td>
  </tr>
  <tr>
    <td >1</td>
    <td >0</td>
    <td >1</td>
    <td >1</td>
  </tr>
  <tr>
    <td >1</td>
    <td >1</td>
    <td >1</td>
    <td >0</td>
  </tr>
</table>

In [3]:
def XOR(input1, input2):
    """
    Takes as input two strings representing either a binary 0 or 1,
    returns their sum modulo 2.
    
    input1 (string): a binary number representing a bit.
    input2 (string): a binary number representing a bit.
    returns(int): the sum modulo two of the two bitstrings.
    """
    q = QuantumRegister(2) # two qubits in which to encode and manipulate the input
    c = ClassicalRegister(1) # a bit to store the output
    qc = QuantumCircuit(q, c) # this is where the quantum program goes

    # Since the qubits are initially in the |0⟩ state, we don't need to do anything for an input of '0'
    # For either input that is '1', we flip the corresponding qubit from |0⟩ to |1⟩
    if input1 == '1':
        qc.x(q[0])
    if input2 == '1':
        qc.x(q[1])
    
    # As explained above, we apply the CNOT (cx) gate on input1 and input2, and measure the second qubit.
    qc.cx(q[0],q[1])
    
    qc.measure(q[1],c[0]) # output from qubit 2 is measured
    # We'll run the program on a simulator
    backend = Aer.get_backend('qasm_simulator')
    # Since the output will be deterministic, we can use just a single shot to get it
    job = execute(qc,backend,shots=1,memory=True)
    output = job.result().get_memory()[0]
    
    return output

## AND gate 
Takes two binary strings as input and gives one as output.

The output is '1' only when both the inputs are '1'.

The Toffoli gate, defined on inputs $(q_0,q_1,q_2)$ as $(q_0,q_1,(q_0\:AND\:q_1)XOR(q_2))$.

Note that the output in the third qubit $(q_0\:AND\:q_1)XOR(q_2)$ is the same as $q_0\:AND\:q_1$ when $q_2=|0⟩$, so if we set our third qubit to |0⟩, we can implement the AND gate on the first two inputs.

The following is the logic table for the Toffoli gate, for the cases where the third qubit input is zero.
<table>
  <tr>
    <th colspan="3">Input</th>
    <th colspan="3">Toffoli Output</th>
  </tr>
  <tr>
    <td>q<sub>0</sub></td>
    <td>q<sub>1</sub></td>
    <td>q<sub>2</sub></td>
    <td>q<sub>0</sub></td>
    <td>q<sub>1</sub></td>
    <td>q<sub>2</sub></td>
  </tr>
   <tr>
    <td>0</td>
    <td>0</td>
    <td>0</td>
    <td>0</td>
    <td>0</td>
    <td>0</td>
  </tr>
  <tr>
    <td>0</td>
    <td>1</td>
    <td>0</td>
    <td>0</td>
    <td>1</td>
    <td>0</td>
  </tr>
  <tr>
    <td>1</td>
    <td>0</td>
    <td>0</td>
    <td>1</td>
    <td>0</td>
    <td>0</td>
  </tr>
  <tr>
    <td>1</td>
    <td>1</td>
    <td>0</td>
    <td>1</td>
    <td>1</td>
    <td>1</td>
  </tr>
</table>

In [4]:
def AND(input1,input2):
    """
    Takes as input two strings representing either a binary 0 or 1,
    returns 0 if at least one of them is 0, and 1 otherwise.
    
    input1 (string): a binary number representing a bit.
    input2 (string): a binary number representing a bit.
    returns(int): the result of input1 AND input2.
    """
    q = QuantumRegister(3) # two qubits in which to encode the input, and one for the output
    c = ClassicalRegister(1) # a bit to store the output
    qc = QuantumCircuit(q, c) # this is where the quantum program goes

    # Since the qubits are initially in the |0⟩ state, we don't need to do anything for an input of '0'
    # For either input that is '1', we flip the corresponding qubit from |0⟩ to |1⟩
    if input1 == '1':
        qc.x(q[0])
    if input2 == '1':
        qc.x(q[1])
    # We apply the Toffoli gate on the three qubits, where q[2] is in state |0⟩
    qc.ccx(q[0],q[1],q[2])
    qc.measure(q[2],c[0]) # output from qubit 3 is measured
    
    # We'll run the program on a simulator
    backend = Aer.get_backend('qasm_simulator')
    # Since the output will be deterministic, we can use just a single shot to get it
    job = execute(qc, backend,shots=1,memory=True)
    output = job.result().get_memory()[0]
    
    return output

## NAND gate 
Takes two binary strings as input and gives one as output.

The output is '0' only when both the inputs are '1'.

The NAND gate will be implemented using the Toffoli and the X (NOT) gates, by simply applying the Toffoli gate on the three qubits, where the third qubit is in state |0⟩, and then applying the X gate on the third qubit.

In [5]:
def NAND(input1,input2):
    """
    Takes as input two strings representing either a binary 0 or 1,
    returns 1 if at least one of them is 0, and 0 otherwise (a composition of AND and NOT).
    
    input1 (string): a binary number representing a bit.
    input2 (string): a binary number representing a bit.
    returns(int): the result of NOT(input1 AND input2).
    """
    q = QuantumRegister(3) # two qubits in which to encode the input, and one for the output
    c = ClassicalRegister(1) # a bit to store the output
    qc = QuantumCircuit(q, c) # this is where the quantum program goes
    
    # Since the qubits are initially in the |0⟩ state, we don't need to do anything for an input of '0'
    # For either input that is '1', we flip the corresponding qubit from |0⟩ to |1⟩
    if input1 == '1':
        qc.x(q[0])
    if input2 == '1':
        qc.x(q[1])
    # We first apply the Toffoli gate on the three qubits, where q[2] is in state |0⟩   
    qc.ccx(q[0],q[1],q[2])
    # Then, we apply the X gate on q[2], where q[2] is the result of q[0] AND q[1] 
    qc.x(q[2])
    qc.measure(q[2],c[0]) # output from qubit 3 is measured
    
    # We'll run the program on a simulator
    backend = Aer.get_backend('qasm_simulator')
    # Since the output will be deterministic, we can use just a single shot to get it
    job = execute(qc,backend,shots=1,memory=True)
    output = job.result().get_memory()[0]
    
    return output

## OR gate 
Takes two binary strings as input and gives one as output.

The output is '1' if either input is '1'.

OR can be implemented by first applying the AND gate on the two inputs, second, applying the XOR gate on the two inputs, and lastly, applying the XOR gate on the results of the previous two gates.

Here is a logic table to demonstrate this sequence of operations.

<table>
  <tr>
    <th>q<sub>0</sub></th>
    <th>q<sub>1</sub></th>
    <th>q<sub>0</sub> AND q<sub>1</sub></th>
    <th>q<sub>0</sub> XOR q<sub>1</sub></th>
    <th>(q<sub>0</sub> AND q<sub>1</sub>) XOR (q<sub>0</sub> XOR q<sub>1</sub>)</th>
  </tr>
  <tr>
    <td>0</td>
    <td>0</td>
    <td>0</td>
    <td>0</td>
    <td>0</td>
  </tr>
  <tr>
    <td>0</td>
    <td>1</td>
    <td>0</td>
    <td>1</td>
    <td>1</td>
  </tr>
  <tr>
    <td>1</td>
    <td>0</td>
    <td>0</td>
    <td>1</td>
    <td>1</td>
  </tr>
  <tr>
    <td>1</td>
    <td>1</td>
    <td>1</td>
    <td>0</td>
    <td>1</td>
  </tr>
</table>

In [6]:
def OR(input1, input2):
    """
    Takes as input two strings representing either a binary 0 or 1,
    returns 1 if at least one of them is 1, and 0 otherwise (a composition of AND and XOR).
    
    input1 (string): a binary number representing a bit.
    input2 (string): a binary number representing a bit.
    returns(int): the result of (input1 AND input2) XOR (input1 XOR input2).
    """
    q = QuantumRegister(3) # two qubits in which to encode the input, and one for the output
    c = ClassicalRegister(1) # a bit to store the output
    qc = QuantumCircuit(q, c) # this is where the quantum program goes
    
    # Since the qubits are initially in the |0⟩ state, we don't need to do anything for an input of '0'
    # For either input that is '1', we flip the corresponding qubit from |0⟩ to |1⟩
    if input1 == '1':
        qc.x(q[0])
    if input2 == '1':
        qc.x(q[1])
    
    # Note that we cannot apply the cx gate before the cxx gate, because cx will alter the second qubit.
    # So first we apply the cxx gate (AND), whose result is in the third qubit, then we apply the cx gate (XOR), 
    # whose result is in the second qubit, then we apply the cx gate (XOR) on the second and third qubits,
    # storing the result in the third qubit.
    qc.ccx(q[0],q[1],q[2])
    qc.cx(q[0],q[1])
    qc.cx(q[1],q[2])
    
    qc.measure(q[2], c[0]) # output from qubit 3 is measured
    
    # We'll run the program on a simulator
    backend = Aer.get_backend('qasm_simulator')
    # Since the output will be deterministic, we can use just a single shot to get it
    job = execute(qc,backend,shots=1,memory=True)
    output = job.result().get_memory()[0]
    
    return output

## FANOUT gate

Takes one binary string as input and gives two copies as output.

In [7]:
def FANOUT(input):
    """
    Takes as input a string representing either a binary 0 or 1,
    returns its binary complement
    
    input (string): a binary number representing a bit
    returns(int): two copies of the input bitstring. 
    """
    q = QuantumRegister(2) # two qubits in which to encode the input, and one for the output
    c = ClassicalRegister(2) # a bit to store the output
    qc = QuantumCircuit(q, c) # this is where the quantum program goes
    
    # Since the qubits are initially in the |0⟩ state, we don't need to do anything for an input of '0'
    # For input of '1', we flip the two qubits from |0⟩ to |1⟩ (since we need to measure two qubits for)
    # the copy of the input.
    if input == '1':
        qc.x(q[0])
        qc.x(q[1])
    # we measure the output from both qubits
    qc.measure(q[0],c[0]) 
    qc.measure(q[1],c[1])
    
    # We'll run the program on a simulator
    backend = Aer.get_backend('qasm_simulator')
    # Since the output will be deterministic, we can use just a single shot to get it
    job = execute(qc,backend,shots=1,memory=True)
    output = job.result().get_memory()[0]
    
    return output

## Tests 
The following code runs the functions above for all possible inputs, so that you can check whether they work.

In [8]:
print('\nResults for the NOT gate')
for input in ['0','1']:
    print('    Input',input,'gives output',NOT(input))
    
print('\nResults for the XOR gate')
for input1 in ['0','1']:
    for input2 in ['0','1']:
        print('    Inputs',input1,input2,'give output',XOR(input1,input2))

print('\nResults for the AND gate')
for input1 in ['0','1']:
    for input2 in ['0','1']:
        print('    Inputs',input1,input2,'give output',AND(input1,input2))

print('\nResults for the NAND gate')
for input1 in ['0','1']:
    for input2 in ['0','1']:
        print('    Inputs',input1,input2,'give output',NAND(input1,input2))

print('\nResults for the OR gate')
for input1 in ['0','1']:
    for input2 in ['0','1']:
        print('    Inputs',input1,input2,'give output',OR(input1,input2))

print('\nResults for the FANOUT gate')
for input in ['0','1']:
    print('    Input',input,'gives output',FANOUT(input))


Results for the NOT gate
    Input 0 gives output 1
    Input 1 gives output 0

Results for the XOR gate
    Inputs 0 0 give output 0
    Inputs 0 1 give output 1
    Inputs 1 0 give output 1
    Inputs 1 1 give output 0

Results for the AND gate
    Inputs 0 0 give output 0
    Inputs 0 1 give output 0
    Inputs 1 0 give output 0
    Inputs 1 1 give output 1

Results for the NAND gate
    Inputs 0 0 give output 1
    Inputs 0 1 give output 1
    Inputs 1 0 give output 1
    Inputs 1 1 give output 0

Results for the OR gate
    Inputs 0 0 give output 0
    Inputs 0 1 give output 1
    Inputs 1 0 give output 1
    Inputs 1 1 give output 1

Results for the FANOUT gate
    Input 0 gives output 00
    Input 1 gives output 11


In [9]:
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.12.0',
 'qiskit-aer': '0.4.1',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.5.0',
 'qiskit-aqua': '0.6.4',
 'qiskit': '0.16.1'}